In [1]:
# 导入必要的包
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit

from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers
from keras.optimizers import Adam
from keras.optimizers import rmsprop
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

from keras.datasets import cifar10
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception
from keras.applications.mobilenet import MobileNet
from keras.applications.vgg19 import VGG19
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

Using TensorFlow backend.


In [2]:
## 依据已有图片做随机旋转，生成更多样本数据

# 使用ImageDataGenerator处理图片
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

### 数据预处理部分

In [3]:
# 生成更多的样本
def GetMore_Image(cup_type):
    if cup_type == 'd':
        print "Processing {}".format(cup_type)
        
        for i in range(1,10):
            img = load_img('./original_data/cup_{c_type}/{c_type}{num}.png'.format(c_type = cup_type, num = i))
            x = img_to_array(img) 
            x = x.reshape((1,) + x.shape) 
            
            a = 0
            for batch in datagen.flow(x, 
                                      batch_size=1, 
                                      save_to_dir='/Users/Kum/Desktop/Machine_Learning/project/cup_classification/processed_data/', 
                                      save_prefix='{c_type}_cup'.format(c_type = cup_type), 
                                      save_format='jpeg'):
                a += 1
                if a > 20:
                    break
            
            print "Cup {} of {} is Done!".format(cup_type, i)
    else:
        print "Processing {}".format(cup_type)
        
        for i in range(1,11):
            img = load_img('./original_data/cup_{c_type}/{c_type}{num}.png'.format(c_type = cup_type, num = i))
            x = img_to_array(img) 
            x = x.reshape((1,) + x.shape) 
            
            a = 0
            for batch in datagen.flow(x, 
                                      batch_size=1, 
                                      save_to_dir='/Users/Kum/Desktop/Machine_Learning/project/cup_classification/processed_data/', 
                                      save_prefix='{c_type}_cup'.format(c_type = cup_type), 
                                      save_format='jpeg'):
                a += 1
                if a > 20:
                    break
            
            print "Cup {} of {} is Done!".format(cup_type, i)  


In [17]:
# 存在一个文件夹
for cup_type in ['d']:
    GetMore_Image('{}'.format(cup_type))

Processing d
Cup d of 1 is Done!
Cup d of 2 is Done!
Cup d of 3 is Done!
Cup d of 4 is Done!
Cup d of 5 is Done!
Cup d of 6 is Done!
Cup d of 7 is Done!
Cup d of 8 is Done!
Cup d of 9 is Done!


### 模型训练部分

In [3]:
# 设定必要的变量
img_width, img_height = 300, 300
data_dir = './processed_data/'
epochs = 50
batch_size = 8
input_shape = (img_width, img_height, 3)

In [4]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [8]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 0 images belonging to 0 classes.
